In [1]:
# imports and setup
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse
from PIL import Image

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [2]:
# reading image into array
hdu = fits.open("filename")
data = hdu[0].data

FileNotFoundError: [Errno 2] No such file or directory: 'filename'

In [ ]:
# showing the image and saving it
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('original.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# measuring the image background
bkg = sep.Background(data)

In [ ]:
# printing the "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluating the background as 2-d array
bkg_image = bkg.back()

In [ ]:
# showing the background and saving it
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('background.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# evaluating the background noise as 2-d array
bkg_rms = bkg.rms()

In [ ]:
# showing the background noise and saving it
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('backgroundnoise.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# subtracting the background from the data
data_sub = data - bkg

In [ ]:
# detecting amount of objects in the no background image
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
len(objects)

In [ ]:
# plotting background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plotting an ellipse that shows each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
# saving final version
plt.savefig('objectsshown.png', bbox_inches = "tight", dpi = 600)

In [ ]:
# available fields
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# showing the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))